<a href="https://www.kaggle.com/code/pogscafe/era3d-kaggle?scriptVersionId=183143481" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Era 3D notebook for Kaggle

Session options:
* Accelerator: GPU T4 x2
* Persistence: Variables and Files
* Environment: Pin to original
* Internet on

How-to video:  
<a href="https://youtu.be/1jcSA5TjZ1k"><img src="https://i3.ytimg.com/vi/1jcSA5TjZ1k/maxresdefault.jpg" width=200) /></a>


Era3D on GitHub: https://github.com/pengHTYX/Era3D

In [ ]:
%%time
home_dir = '/kaggle/working'
python = '/kaggle/working/venv/bin/python'
pip = '/kaggle/working/venv/bin/pip'

download_venv = True

import os
import stat

if (not os.path.exists(f'{home_dir}/venv')) and download_venv == True:
    get_ipython().system('wget "https://j2q5.c17.e2-1.dev/download/pogscafe/era3d_kaggle/venv.tar.gz" -O "/kaggle/venv.tar.gz"')
    get_ipython().system('tar -xvf /kaggle/venv.tar.gz -C /kaggle/working/')
    
    
!git lfs install

if not os.path.exists(f'{home_dir}/venv/bin/python3.10'):
    get_ipython().system('cp /opt/conda/bin/python3.10 /kaggle/working/venv/bin/')
!ln -s /kaggle/working/venv/bin/python3.10 /kaggle/working/venv/bin/python
!ln -s /kaggle/working/venv/bin/python3.10 /kaggle/working/venv/bin/python3

def find_bin_folders(folder_path):
    bin_folders = []
    for root, dirs, files in os.walk(folder_path):
        for dir_name in dirs:
            if dir_name == 'bin':
                bin_folders.append(os.path.join(root, dir_name)) 
    return bin_folders

def installEra(home_dir):
  %cd {home_dir}
  !git lfs clone https://huggingface.co/spaces/pengHTYX/Era3D_MV_demo/
  %cd {home_dir}/Era3D_MV_demo
  !git clone https://github.com/wandaweb/Era3D
  !mv {home_dir}/Era3D_MV_demo/Era3D/instant-nsr-pl {home_dir}/Era3D_MV_demo/instant-nsr-pl
  !mv {home_dir}/Era3D_MV_demo/Era3D/blender {home_dir}/Era3D_MV_demo/
  !mv {home_dir}/Era3D_MV_demo/Era3D/data_lists {home_dir}/Era3D_MV_demo/
  !mv {home_dir}/Era3D_MV_demo/Era3D/node_config {home_dir}/Era3D_MV_demo/
  os.makedirs('/kaggle/temp', exist_ok=True)
  %cd /kaggle/temp/
  !git lfs clone https://huggingface.co/pengHTYX/MacLab-Era3D-512-6view
  !rm -rf /kaggle/working/Era3D_MV_demo/MacLab-Era3D-512-6view  
  !ln -s /kaggle/temp/MacLab-Era3D-512-6view /kaggle/working/Era3D_MV_demo/

def installLibraries(home_dir, python, pip):
  %cd {home_dir}
  !{python} -m pip install --force-reinstall torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 xformers --index-url https://download.pytorch.org/whl/cu118
  !{pip} install pygments
  if not os.path.exists(f'tinycudann-1.7-cp310-cp310-linux_x86_64.whl'):
    get_ipython().system('wget "https://j2q5.c17.e2-1.dev/download/pogscafe/tinycudann-1.7-cp310-cp310-linux_x86_64.whl"')
  !{pip} install tinycudann-1.7-cp310-cp310-linux_x86_64.whl
  !rm -rf 
  !{pip} install git+https://github.com/NVlabs/nvdiffrast
  %cd {home_dir}/Era3D_MV_demo/
  !{pip} install -r requirements.txt
  !{pip} install fire spaces
  !{pip} install moviepy trimesh rembg icecream
  !rm -rf /opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info
  %cd {home_dir}/Era3D_MV_demo/Era3D/instant-nsr-pl
  !{pip} install -r requirements.txt
  !{pip} install --force-reinstall numpy
  !{pip} install --force-reinstall pydantic
  !{pip} install tensorflow[and-cuda]

get_ipython().system('pip install virtualenv')
installEra(home_dir)
if not os.path.exists(f'{home_dir}/venv'):
    print('installing venv')
    venv_parent = home_dir
    print(venv_parent)
    os.chdir(venv_parent)
    get_ipython().system(f'cd {venv_parent}')
    
    get_ipython().system('virtualenv venv -p $(which python3.10)')
    installLibraries(home_dir, python, pip)
else:
    bin_folders = find_bin_folders('/kaggle/working/venv')
    if bin_folders:
      print("Found 'bin' folders:")
      for bin_folder in bin_folders:
        print(bin_folder)
        for filename in os.listdir(bin_folder):
            file_path = os.path.join(bin_folder, filename)
            if os.path.isfile(file_path):
                current_permissions = os.stat(file_path).st_mode
                 # Add execute permissions for the user, group, and others
                os.chmod(file_path, current_permissions | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)

!apt install -y mesa-utils freeglut3-dev libnvidia-gl-470 libegl1-mesa-dev

import fileinput
for line in fileinput.input(f'{home_dir}/Era3D_MV_demo/app.py', inplace=True):
    if ('demo.queue().launch(share=True, max_threads=80)' in line):
       line = line.replace('demo.queue().launch(share=True, max_threads=80)', 'demo.queue().launch(share=False, max_threads=20)')
    print(line, end='')

os.environ["PATH"] += ":/kaggle/working/venv/lib/python3.10"

# Start the WebUI
You can use the WebUI to generate images and normals for mesh creation.

## Start with Pinggy
Start the code cell, wait for the app to start, and follow the link ending with .pinggy.link.

In [ ]:
# Starting the Web UI with pinggy

%cd /kaggle/working/Era3D_MV_demo/

from multiprocessing import Process
import sys
import time

open('log.txt', 'w').close()

def run_app():
    cmd = f"{python} /kaggle/working/Era3D_MV_demo/app.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:7860 a.pinggy.io | tee log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("")
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

# Generate the Mesh

In [ ]:
%%time
# Leave the input path empty to use the latest generated images, or enter the full path 
# to the images you want turned into a mesh.
input_path = '' #@param {type:"string"}
# Example: input_path = "/kaggle/working/Era3D_MV_demo/mv_res/cropsize-420-cfg3.0/scene@20240605-211017"

# --------------------------------------------------

import os

def find_highest_second_level_subfolder(path):
    highest_folder = None
    parent_folder = None
    for root, dirs, files in os.walk(path):
        depth = root[len(path):].count(os.sep)
        if depth == 1:
            for d in dirs:
                if highest_folder is None or d > highest_folder:
                    highest_folder = d
                    parent_folder = root
    return highest_folder, parent_folder

input_dir = ''
if input_path == '':
  mv_res_path = '/kaggle/working/Era3D_MV_demo/mv_res'
  highest_folder, parent_folder = find_highest_second_level_subfolder(mv_res_path)
  input_path=os.path.join(parent_folder, highest_folder)
  print(input_path)
input_dir = input_path.split('mv_res/')[1]
outdir = '/kaggle/working/output'
import os
print(input_dir)
os.makedirs(outdir, exist_ok=True)
%cd /kaggle/working/Era3D_MV_demo/instant-nsr-pl
!{python} launch.py  --config configs/neuralangelo-ortho-wmask.yaml --gpu 0 \
    --train dataset.root_dir=../mv_res dataset.scene={input_dir} \
    --exp_dir={outdir}
print('Finished')

# File browser

### Install FileBrowser

In [ ]:
# Install FileBrowser
%cd /kaggle/working
!wget https://github.com/filebrowser/filebrowser/releases/download/v2.27.0/linux-amd64-filebrowser.tar.gz
!tar xvfz linux-amd64-filebrowser.tar.gz
!chmod a+x /kaggle/working/filebrowser
!rm /kaggle/working/config.json
!/kaggle/working/filebrowser config init 
!/kaggle/working/filebrowser config set --auth.method=noauth > /dev/null
!/kaggle/working/filebrowser config set --branding.theme=dark > /dev/null
!/kaggle/working/filebrowser users add admin admin > /dev/null
!/kaggle/working/filebrowser config export "/kaggle/working/config.json" > /dev/null

### Start the File Browser
Follow the link ending with .pinggy.link to open the file browser.

In [ ]:
# Start FileBrowser

%cd /kaggle/working
!chmod a+x /kaggle/working/filebrowser
        
from multiprocessing import Process
import sys
import time

open('log.txt', 'w').close()

def run_app():
    !/kaggle/working/filebrowser -c "/kaggle/working/config.json" & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8080 a.pinggy.io > log.txt > log.txt
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

### Re-initialize variables

In [ ]:
# Re-initialize variables (if you restarted the runtime)
home_dir = '/kaggle/working'
python = '/kaggle/working/venv/bin/python'
pip = '/kaggle/working/venv/bin/pip'

### Delete all outputs to free up space

In [ ]:
!rm -rf /kaggle/working/output/*
!rm -rf /kaggle/working/Era3D_MV_demo/mv_res/*

### Clear the working folder

In [ ]:
# Delete everything
!rm -rf /kaggle/working/*
!rm -rf /kaggle/temp/*

# 